## 1º Bloco — Event Preprocessing

In [9]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import csv
import re

import sparse  # pydata/sparse

In [15]:
from pathlib import Path

# You are currently here:
# .../Tese/dev/xR-EvT-new/dataset_scripts
NOTEBOOK_CWD = Path.cwd()

# Go back to .../Tese
PROJECT_ROOT = NOTEBOOK_CWD.parents[3]  # Tese
DVS_ROOT = PROJECT_ROOT / "datasets" / "DVS128"
DATA = DVS_ROOT / "data"

TRIAL_STEM = "user01_fluorescent_led"
AEDAT_PATH  = DATA / f"{TRIAL_STEM}.aedat"
LABELS_PATH = DATA / f"{TRIAL_STEM}_labels.csv"

print("PROJECT_ROOT:", PROJECT_ROOT)
print("AEDAT exists:", AEDAT_PATH.exists(), AEDAT_PATH)
print("LABELS exists:", LABELS_PATH.exists(), LABELS_PATH)


PROJECT_ROOT: /Users/pedrofs/ISCTE-Projetos/Mestrado
AEDAT exists: False /Users/pedrofs/ISCTE-Projetos/Mestrado/datasets/DVS128/data/user01_fluorescent_led.aedat
LABELS exists: False /Users/pedrofs/ISCTE-Projetos/Mestrado/datasets/DVS128/data/user01_fluorescent_led_labels.csv


In [ ]:
# === Dataset paths (edit if needed) ===
ROOT = Path("datasets/DVS128")          # parent folder where gesture_mapping.csv + README.txt live
DATA = ROOT / "data"                    # folder containing .aedat trials

TRIAL_STEM = "user01_fluorescent_led"   # chosen trial
AEDAT_PATH  = DATA / f"{TRIAL_STEM}.aedat"
LABELS_PATH = DATA / f"{TRIAL_STEM}_labels.csv"  # if yours is in ROOT instead, change to ROOT / f"{TRIAL_STEM}_labels.csv"

# === DVS128 sensor resolution ===
H, W = 128, 128

# === xR-EvT sparse frame duration (from folder name clean_dataset_frames_12000) ===
SPARSE_FRAME_LEN_US = 12000  # 12ms

# === Model-facing Δt (Event_DataModule.chunk_len_ms in xR-EvT) ===
CHUNK_LEN_MS = 48   # set to your training config
BINS = 4            # set to your training config (bins)

# === Polarity preprocessing in EventDataset: merge or keep 2 channels ===
MERGE_POLARITIES = False  # if True -> preproc_event_size = 1, else 2

### Read .aedat polarity events into [N,4] = (x,y,t,p)

This implements the AEDAT 3.1 polarity decoding described in the README:
- Each event: uint32 data, uint32 timestamp
- x = (data >> 17) & 0x1FFF
- y = (data >> 2) & 0x1FFF
- p = (data >> 1) & 0x1 (0/1)
- t = timestamp (µs)

In [11]:
def read_aedat31_polarity_events(path: Path) -> np.ndarray:
    with path.open("rb") as f:
        data_bytes = f.read()

    # AEDAT 3.1 file consists of repeated [header][events] blocks.
    # Header fields (8 total):
    # uint16 eventType, uint16 eventSource,
    # uint32 eventSize, uint32 eventTSOffset, uint32 eventTSOverflow,
    # uint32 eventCapacity, uint32 eventNumber, uint32 eventValid
    offset = 0
    events_list = []

    while offset + 28 <= len(data_bytes):
        # Read header
        eventType   = int.from_bytes(data_bytes[offset:offset+2], "little")
        eventSource = int.from_bytes(data_bytes[offset+2:offset+4], "little")
        eventSize   = int.from_bytes(data_bytes[offset+4:offset+8], "little")
        eventTSOff  = int.from_bytes(data_bytes[offset+8:offset+12], "little")
        eventTSOv   = int.from_bytes(data_bytes[offset+12:offset+16], "little")
        eventCap    = int.from_bytes(data_bytes[offset+16:offset+20], "little")
        eventNum    = int.from_bytes(data_bytes[offset+20:offset+24], "little")
        eventValid  = int.from_bytes(data_bytes[offset+24:offset+28], "little")
        offset += 28

        # Defensive checks
        if eventNum == 0 or eventSize == 0:
            continue

        block_bytes = eventNum * eventSize
        if offset + block_bytes > len(data_bytes):
            break

        # Polarity events in this dataset are stored as [uint32 data][uint32 timestamp]
        # eventSize should be 8 for polarity events here.
        raw = np.frombuffer(data_bytes, dtype=np.uint32, count=(block_bytes // 4), offset=offset)
        offset += block_bytes

        if raw.size < 2:
            continue

        # Interpret as pairs: (data, timestamp)
        raw = raw.reshape(-1, 2)
        data_word = raw[:, 0]
        ts = raw[:, 1]

        x = (data_word >> 17) & 0x1FFF
        y = (data_word >> 2)  & 0x1FFF
        p = (data_word >> 1)  & 0x1

        # Keep only events within sensor bounds (extra safety)
        m = (x < W) & (y < H)
        x, y, ts, p = x[m], y[m], ts[m], p[m]

        # Store as (x,y,t,p)
        ev = np.stack([x, y, ts, p], axis=1).astype(np.int64)
        events_list.append(ev)

    if not events_list:
        raise RuntimeError(f"No events parsed from {path}")

    events = np.concatenate(events_list, axis=0)
    # Ensure time-sorted (most pipelines rely on it)
    events = events[np.argsort(events[:, 2])]
    return events

### Load labels CSV and pick ONE gesture segment

Each CSV row is a gesture instance: 
- class,startTime_usec,endTime_usec

In [12]:
def read_labels_csv(path: Path):
    rows = []
    with path.open("r", newline="") as f:
        reader = csv.DictReader(f)
        for r in reader:
            rows.append({
                "class": int(r["class"]),
                "startTime_usec": int(r["startTime_usec"]),
                "endTime_usec": int(r["endTime_usec"]),
            })
    return rows

In [13]:
labels = read_labels_csv(LABELS_PATH)
len(labels), labels[0]

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/DVS128/data/user01_fluorescent_led_labels.csv'

In [ ]:
IDX = 0 

gesture = labels[IDX]
c = gesture["class"]
t0 = gesture["startTime_usec"]
t1 = gesture["endTime_usec"]
(c, t0, t1)

In [ ]:
def slice_events_by_time(events_xypt: np.ndarray, t0: int, t1: int) -> np.ndarray:
    t = events_xypt[:, 2]
    m = (t >= t0) & (t < t1)
    return events_xypt[m]

events_g = slice_events_by_time(events_all, t0, t1)
events_g.shape

### Events → 12ms sparse frames (T_sf, H, W, 2) (counts, uint8, clipped)

Group into non-overlapping windows of 12000µs, build frames (H,W,2) with indices [y,x,polarity] and values = counts clipped to [0,255] stored as uint8.

In [ ]:
def events_to_sparse_frames_12ms(
    events_xypt: np.ndarray,
    H: int,
    W: int,
    frame_len_us: int = 12000,
) -> sparse.COO:
    """
    Convert events (x,y,t,p) into stacked sparse frames:
      output shape: (T_sf, H, W, 2)
    Values are event counts per pixel/polarity, clipped to [0,255] and stored as uint8.
    """
    if events_xypt.size == 0:
        # return an empty stack
        return sparse.COO(coords=np.zeros((4, 0), dtype=np.int64),
                          data=np.zeros((0,), dtype=np.uint8),
                          shape=(0, H, W, 2))

    x = events_xypt[:, 0].astype(np.int64)
    y = events_xypt[:, 1].astype(np.int64)
    t = events_xypt[:, 2].astype(np.int64)
    p = events_xypt[:, 3].astype(np.int64)

    t_start = int(t.min())
    t_end   = int(t.max()) + 1

    # Define non-overlapping windows [ws, ws+frame_len_us)
    # Number of frames covering [t_start, t_end)
    T_sf = int(np.ceil((t_end - t_start) / frame_len_us))

    # Compute frame index for each event
    fidx = (t - t_start) // frame_len_us
    fidx = np.clip(fidx, 0, T_sf - 1)

    # Accumulate counts into COO coords (f, y, x, p)
    coords = np.stack([fidx, y, x, p], axis=0)  # [4, N]

    # Count duplicates: use sparse.COO with ones then sum duplicates
    data = np.ones((coords.shape[1],), dtype=np.int32)
    coo = sparse.COO(coords=coords, data=data, shape=(T_sf, H, W, 2))
    coo = coo.sum_duplicates()

    # Clip and cast to uint8
    coo_data = np.clip(coo.data, 0, 255).astype(np.uint8)
    coo = sparse.COO(coords=coo.coords, data=coo_data, shape=coo.shape)
    return coo

In [ ]:
sf = events_to_sparse_frames_12ms(events_g, H=H, W=W, frame_len_us=SPARSE_FRAME_LEN_US)
sf.shape, sf.dtype

In [ ]:
frame_id = min(0, sf.shape[0]-1)
dense0 = sf[frame_id].todense()  # (H, W, 2) uint8
img0 = dense0.sum(axis=-1)       # (H, W)

plt.figure()
plt.imshow(img0)
plt.title(f"Sparse frame #{frame_id} (sum over polarity), shape={dense0.shape}")
plt.axis("off")
plt.show()